In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
from selenium import webdriver
from tqdm.notebook import tqdm

## 오픈갤러리

In [58]:
data=pd.DataFrame(columns=['title','artists','location','term','open_at','close_at','day','explain','price'])

home=requests.get('https://www.opengallery.co.kr/exhibition/?flt_region=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C')

html=home.text
home_soup=BeautifulSoup(html, 'html.parser')

exhibitions=home_soup.select('div.exhibitionCard > a.exhibitionCard-a')
ex_links=["https://www.opengallery.co.kr"+exhibitions[i].get('href') for i in range(len(exhibitions))]

for i in range(len(ex_links)):
    page=requests.get(ex_links[i])
    page_html=page.text

    page_soup=BeautifulSoup(page_html, 'html.parser')

    #<--info
    title=page_soup.select('h2.exhibitionDetail-head-title')[0].text

    artists=page_soup.select('td.exhibitionDetail-infoTable-artists')[0].text

    info_table=page_soup.select('table.exhibitionDetail-infoTable-table td')

    location = page_soup.select('div.exhibitionDetail-sm')[-1].text
    term = info_table[2].text
    open_at = info_table[3].text
    close_at = info_table[3].text
    day = info_table[3].text
#     site = page_soup.select('td.exhibitionDetail-infoTable-url a')[0].get('href')
    try:
        explain=page_soup.select('div.exhibitionDetail-explain')[0].text
    except:
        explain="none"
    price = info_table[4].text
    #info-->

    data.loc[i]=[title, artists, location, term, open_at, close_at, day, explain, price]

In [60]:
data.to_csv('opengallery_raw.csv')

In [59]:
data

,title,artists,location,term,open_at,close_at,day,explain,price
0,《VERT ART》 ARTIST TALK 유튜브 동시중계 (4/18 SAT),\nSen Chung\n,서울특별시 종로구 우정국로 45-14 (수송동),\n 2020-04-18 ~...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,​OCI미술관에서 샌정 초대개인전 《VERY ART》의 아티스트 토크를 진행합니다....,\n \n ...
1,모네에서 세잔까지,"\n폴 세잔,\n \...",서울특별시 서초구 남부순환로 2406 (서초동) 예술의전당 한가람미술관,\n 2020-01-17 ~...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,이번 ‘모네에서 세잔까지: 국립이스라엘 미술관 인상파 걸작展’은 전시와 문화 예술의...,\n \n ...
2,9 to 0 (Nine to Zero),\n요요진\n,서울특별시 마포구 독막로9길 34 (서교동),\n 2020-04-03 ~...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,"[전시요약]아트아치, 2020.04.03 ~ 2020.04.19선과 선을 이어 독특...",\n \n ...
3,볼로냐 일러스트 원화전 2019,\n \n ...,서울특별시 서초구 남부순환로 2406 (서초동) 예술의전당 한가람미술관,\n 2020-02-06 ~...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,<볼로냐 일러스트 원화전>은 1967년부터 시작하여 2019년 53회째 전시를 진행...,\n \n ...
4,그늘이 머물다간 풍경,\n권소영\n,서울특별시 종로구 삼청로7길 37 (팔판동),\n 2020-04-22 ~...,\n 11:00 ~ 18:0...,\n 11:00 ~ 18:0...,\n 11:00 ~ 18:0...,자연이 지닌 태초의 힘 여행지에서의 자연은 일상에서 느낄 수 없는 다양한 영감과 ...,\n \n ...
5,On High,\n이경현\n,서울특별시 중구 을지로5길 16 (을지로2가) 지하 1층,\n 2020-04-06 ~...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,none,\n \n ...
6,임현경 ‘숲의 장막' 展,\n임현경\n,서울특별시 관악구 관악로 1 (신림동),\n 2020-02-04 ~...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,"는 도시 속에 마주하게 되는 공원, 혹은 정원의 풍경에서 수많은 나무와 그것을 가꾸...",\n \n ...
7,강남모던걸,"\n몽상,\n \n ...",서울특별시 강남구 봉은사로 120 (역삼동) 1층,\n 2019-12-20 ~...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,\n 11:00 ~ 20:0...,강남한복판에서 열리는 감성 비주얼 체험전시 <강남모던-걸>展뉴트로(newtro)는 ...,\n \n ...
8,툴루즈 로트렉 展,\n툴루즈 로트렉\n,서울특별시 서초구 남부순환로 2406 (서초동) 예술의전당 한가람미술관,\n 2020-01-14 ~...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,\n 10:00 ~ 19:0...,"<현대 포스터의 아버지>로 불리는 툴루즈 로트렉은 19세기 후반, 예술의 거리 몽...",\n \n ...
9,VERY ART,\nSen Chung\n,서울특별시 종로구 우정국로 45-14 (수송동) OCI 미술관,\n 2020-03-19 ~...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,\n 10:00 ~ 18:0...,"회화적인, 너무도 회화적인캔버스를 마주한 화가에게 그림이란 어떤 의미인가? 수 세기...",\n \n ...


## 왓챠

왓챠에서 '김지빈' 유저가 평가한 영화 목록을 가져온다

In [99]:
url = 'https://watcha.com/ko-KR/users/24BqE2RY8vrjn/contents/movies/ratings'
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)

driver.get(url)

SCROLL_PAUSE_TIME=2.5
last_height = driver.execute_script("return document.body.scrollHeight")

scroll_num = 0

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(SCROLL_PAUSE_TIME)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    
    scroll_num += 1
    last_height = new_height

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

movie_list=soup.select('li.css-106b4k6-Self.e3fgkal0 > a')
movies=["https://watcha.com"+movie_list[i].get('href') for i in range(len(movie_list))]

print(movies)
with open('movie_list_raw.txt', 'w') as file:
    file.write('\n'.join(movies)) #read -> file.readlines()

['https://watcha.com/ko-KR/contents/mQO8625', 'https://watcha.com/ko-KR/contents/mWyJbrM', 'https://watcha.com/ko-KR/contents/mOk6r9z', 'https://watcha.com/ko-KR/contents/mdB7yP5', 'https://watcha.com/ko-KR/contents/mO1Vx4Y', 'https://watcha.com/ko-KR/contents/mAd7lv5', 'https://watcha.com/ko-KR/contents/mWywz9V', 'https://watcha.com/ko-KR/contents/mdKD2aV', 'https://watcha.com/ko-KR/contents/mWyJa0K', 'https://watcha.com/ko-KR/contents/m5ZlXxL', 'https://watcha.com/ko-KR/contents/mWpJvAd', 'https://watcha.com/ko-KR/contents/mOAZ6Rl', 'https://watcha.com/ko-KR/contents/mOPNXw5', 'https://watcha.com/ko-KR/contents/mOPomg7', 'https://watcha.com/ko-KR/contents/m5Q9Zg8', 'https://watcha.com/ko-KR/contents/md7r0l1', 'https://watcha.com/ko-KR/contents/m5NVKVY', 'https://watcha.com/ko-KR/contents/mOP0v0Y', 'https://watcha.com/ko-KR/contents/mdErj22', 'https://watcha.com/ko-KR/contents/m5ZZ2P9', 'https://watcha.com/ko-KR/contents/m5nQrpo', 'https://watcha.com/ko-KR/contents/mOVvVyp', 'https://

In [1]:
movies=['https://watcha.com/ko-KR/contents/mQO8625', 'https://watcha.com/ko-KR/contents/mWyJbrM', 'https://watcha.com/ko-KR/contents/mOk6r9z', 'https://watcha.com/ko-KR/contents/mdB7yP5', 'https://watcha.com/ko-KR/contents/mO1Vx4Y', 'https://watcha.com/ko-KR/contents/mAd7lv5', 'https://watcha.com/ko-KR/contents/mWywz9V', 'https://watcha.com/ko-KR/contents/mdKD2aV', 'https://watcha.com/ko-KR/contents/mWyJa0K', 'https://watcha.com/ko-KR/contents/m5ZlXxL', 'https://watcha.com/ko-KR/contents/mWpJvAd', 'https://watcha.com/ko-KR/contents/mOAZ6Rl', 'https://watcha.com/ko-KR/contents/mOPNXw5', 'https://watcha.com/ko-KR/contents/mOPomg7', 'https://watcha.com/ko-KR/contents/m5Q9Zg8', 'https://watcha.com/ko-KR/contents/md7r0l1', 'https://watcha.com/ko-KR/contents/m5NVKVY', 'https://watcha.com/ko-KR/contents/mOP0v0Y', 'https://watcha.com/ko-KR/contents/mdErj22', 'https://watcha.com/ko-KR/contents/m5ZZ2P9', 'https://watcha.com/ko-KR/contents/m5nQrpo', 'https://watcha.com/ko-KR/contents/mOVvVyp', 'https://watcha.com/ko-KR/contents/mdKBgy9', 'https://watcha.com/ko-KR/contents/md76YRp', 'https://watcha.com/ko-KR/contents/mdRL4eL', 'https://watcha.com/ko-KR/contents/m5ewr3z', 'https://watcha.com/ko-KR/contents/m5XM27B', 'https://watcha.com/ko-KR/contents/mOVDa1p', 'https://watcha.com/ko-KR/contents/mdRLXvW', 'https://watcha.com/ko-KR/contents/mWvkVBx', 'https://watcha.com/ko-KR/contents/mdBzNnP', 'https://watcha.com/ko-KR/contents/mdjwR8l', 'https://watcha.com/ko-KR/contents/mOgoNMO', 'https://watcha.com/ko-KR/contents/mWyJxx1', 'https://watcha.com/ko-KR/contents/mWwlN3O', 'https://watcha.com/ko-KR/contents/m533xYy', 'https://watcha.com/ko-KR/contents/mWqJbZX', 'https://watcha.com/ko-KR/contents/mO0gQ41', 'https://watcha.com/ko-KR/contents/mdRLZVK', 'https://watcha.com/ko-KR/contents/md76rkk', 'https://watcha.com/ko-KR/contents/mLOPBa5', 'https://watcha.com/ko-KR/contents/mY5QZDW', 'https://watcha.com/ko-KR/contents/m5nQwly', 'https://watcha.com/ko-KR/contents/mWJyX7x', 'https://watcha.com/ko-KR/contents/m5xJZE3', 'https://watcha.com/ko-KR/contents/mObGV9n', 'https://watcha.com/ko-KR/contents/mWJyDKM', 'https://watcha.com/ko-KR/contents/mWqJy9K', 'https://watcha.com/ko-KR/contents/m5m1YNG', 'https://watcha.com/ko-KR/contents/mOVv2Lz', 'https://watcha.com/ko-KR/contents/m5awllO', 'https://watcha.com/ko-KR/contents/mOAZkEG', 'https://watcha.com/ko-KR/contents/mdKBpoY', 'https://watcha.com/ko-KR/contents/mdRLbZr', 'https://watcha.com/ko-KR/contents/mWyJa2k', 'https://watcha.com/ko-KR/contents/mWyJYkO', 'https://watcha.com/ko-KR/contents/m5GXo8r', 'https://watcha.com/ko-KR/contents/m5QqAPG', 'https://watcha.com/ko-KR/contents/mdjawmz', 'https://watcha.com/ko-KR/contents/mOVvDjg', 'https://watcha.com/ko-KR/contents/m5Y6yQW', 'https://watcha.com/ko-KR/contents/md7Z6vO', 'https://watcha.com/ko-KR/contents/mdMgNAJ', 'https://watcha.com/ko-KR/contents/mO8KXmX', 'https://watcha.com/ko-KR/contents/m5Y6LNl', 'https://watcha.com/ko-KR/contents/mdEwMrg', 'https://watcha.com/ko-KR/contents/mWyJ8VO', 'https://watcha.com/ko-KR/contents/mWzw6E0', 'https://watcha.com/ko-KR/contents/mOAZkKm', 'https://watcha.com/ko-KR/contents/m5xDb35', 'https://watcha.com/ko-KR/contents/mdRL434', 'https://watcha.com/ko-KR/contents/mOPoAXW', 'https://watcha.com/ko-KR/contents/m5xJxeB', 'https://watcha.com/ko-KR/contents/mWLjDm6', 'https://watcha.com/ko-KR/contents/m5xDab5', 'https://watcha.com/ko-KR/contents/mWvkxbr', 'https://watcha.com/ko-KR/contents/mWyJwbr', 'https://watcha.com/ko-KR/contents/m5DGQDQ', 'https://watcha.com/ko-KR/contents/mdMlb4O', 'https://watcha.com/ko-KR/contents/mOgBBKY', 'https://watcha.com/ko-KR/contents/md6X2aW', 'https://watcha.com/ko-KR/contents/mOVvJ68', 'https://watcha.com/ko-KR/contents/mW4LYVL', 'https://watcha.com/ko-KR/contents/m5QqQ77', 'https://watcha.com/ko-KR/contents/m5m1KM7', 'https://watcha.com/ko-KR/contents/mOl4GK6', 'https://watcha.com/ko-KR/contents/m5GXDep', 'https://watcha.com/ko-KR/contents/mWyJD46', 'https://watcha.com/ko-KR/contents/mO1VKeL', 'https://watcha.com/ko-KR/contents/mOk6oAk', 'https://watcha.com/ko-KR/contents/m5NVpDp', 'https://watcha.com/ko-KR/contents/m5GXzED', 'https://watcha.com/ko-KR/contents/mWwBw83', 'https://watcha.com/ko-KR/contents/mOl4x7g', 'https://watcha.com/ko-KR/contents/mWvkKME', 'https://watcha.com/ko-KR/contents/mWwBwA3', 'https://watcha.com/ko-KR/contents/mWzw7oZ', 'https://watcha.com/ko-KR/contents/mWvkXvZ', 'https://watcha.com/ko-KR/contents/mWyg9kd', 'https://watcha.com/ko-KR/contents/m5eJD2R', 'https://watcha.com/ko-KR/contents/mWpmGjx', 'https://watcha.com/ko-KR/contents/mWLjMPr', 'https://watcha.com/ko-KR/contents/mMO2zkO', 'https://watcha.com/ko-KR/contents/mWywaMd', 'https://watcha.com/ko-KR/contents/mObqb7W', 'https://watcha.com/ko-KR/contents/mW4L6jY', 'https://watcha.com/ko-KR/contents/m5aVMGQ', 'https://watcha.com/ko-KR/contents/mr5ZLlO', 'https://watcha.com/ko-KR/contents/mWw8zEO', 'https://watcha.com/ko-KR/contents/m533vQ2', 'https://watcha.com/ko-KR/contents/mOgBnv0', 'https://watcha.com/ko-KR/contents/m5QKK8O', 'https://watcha.com/ko-KR/contents/mOgb1NW', 'https://watcha.com/ko-KR/contents/mWqJeDX', 'https://watcha.com/ko-KR/contents/md6YvoA', 'https://watcha.com/ko-KR/contents/mO2MNRn', 'https://watcha.com/ko-KR/contents/mO2MNlx', 'https://watcha.com/ko-KR/contents/mWwBb4q', 'https://watcha.com/ko-KR/contents/mgdMnm5', 'https://watcha.com/ko-KR/contents/m5aV96M', 'https://watcha.com/ko-KR/contents/mW93v7e', 'https://watcha.com/ko-KR/contents/mdBzrpD', 'https://watcha.com/ko-KR/contents/mWJypq8', 'https://watcha.com/ko-KR/contents/m5GX0zB', 'https://watcha.com/ko-KR/contents/m5ZXblx', 'https://watcha.com/ko-KR/contents/m5Y6QqQ', 'https://watcha.com/ko-KR/contents/mW93vGe', 'https://watcha.com/ko-KR/contents/mOVvVxY', 'https://watcha.com/ko-KR/contents/mOk60yQ', 'https://watcha.com/ko-KR/contents/mOo0DqZ', 'https://watcha.com/ko-KR/contents/m5NVMAX', 'https://watcha.com/ko-KR/contents/mOg0eNd', 'https://watcha.com/ko-KR/contents/m5eJ1wR', 'https://watcha.com/ko-KR/contents/mObG11N', 'https://watcha.com/ko-KR/contents/mQO8b5R', 'https://watcha.com/ko-KR/contents/m5xJNzl', 'https://watcha.com/ko-KR/contents/m533Nnn', 'https://watcha.com/ko-KR/contents/mOk6ENg', 'https://watcha.com/ko-KR/contents/m5aV9XM', 'https://watcha.com/ko-KR/contents/mOPomng', 'https://watcha.com/ko-KR/contents/m5m1XlB', 'https://watcha.com/ko-KR/contents/mO8KDxX', 'https://watcha.com/ko-KR/contents/mOgBozg', 'https://watcha.com/ko-KR/contents/mOVyqqW', 'https://watcha.com/ko-KR/contents/mdBzP4A', 'https://watcha.com/ko-KR/contents/mObG9n7', 'https://watcha.com/ko-KR/contents/mO8KywG', 'https://watcha.com/ko-KR/contents/mWqJbLo', 'https://watcha.com/ko-KR/contents/m5NVjDa', 'https://watcha.com/ko-KR/contents/md6YDDq', 'https://watcha.com/ko-KR/contents/m5Qq28G', 'https://watcha.com/ko-KR/contents/m5nQD4X', 'https://watcha.com/ko-KR/contents/mWLjM30', 'https://watcha.com/ko-KR/contents/m85XRAO', 'https://watcha.com/ko-KR/contents/mWqJbYB', 'https://watcha.com/ko-KR/contents/mdBzr12', 'https://watcha.com/ko-KR/contents/mWwBMP3', 'https://watcha.com/ko-KR/contents/mW4LxAo', 'https://watcha.com/ko-KR/contents/mOVaaRW', 'https://watcha.com/ko-KR/contents/mObGM6z', 'https://watcha.com/ko-KR/contents/md76JgM', 'https://watcha.com/ko-KR/contents/mO8KDEX', 'https://watcha.com/ko-KR/contents/md71l1W', 'https://watcha.com/ko-KR/contents/mWqJyRK', 'https://watcha.com/ko-KR/contents/mWyJqvy', 'https://watcha.com/ko-KR/contents/md6YDVa', 'https://watcha.com/ko-KR/contents/mObGr9R', 'https://watcha.com/ko-KR/contents/mWLVrpd', 'https://watcha.com/ko-KR/contents/mDWvjEW', 'https://watcha.com/ko-KR/contents/md76J08', 'https://watcha.com/ko-KR/contents/m5ZX9ke', 'https://watcha.com/ko-KR/contents/mdMBAR5', 'https://watcha.com/ko-KR/contents/mOAZjpG', 'https://watcha.com/ko-KR/contents/mdEwMvg', 'https://watcha.com/ko-KR/contents/m5QqpLQ', 'https://watcha.com/ko-KR/contents/mOAZ6Pd', 'https://watcha.com/ko-KR/contents/mOo0PKZ', 'https://watcha.com/ko-KR/contents/mWqJboK', 'https://watcha.com/ko-KR/contents/mMO2GdK', 'https://watcha.com/ko-KR/contents/mWzxGDd', 'https://watcha.com/ko-KR/contents/m5m1v63', 'https://watcha.com/ko-KR/contents/md76DgB', 'https://watcha.com/ko-KR/contents/m5XMMN3', 'https://watcha.com/ko-KR/contents/mdRJqLd', 'https://watcha.com/ko-KR/contents/m533L7p', 'https://watcha.com/ko-KR/contents/mWLjk86', 'https://watcha.com/ko-KR/contents/md76ze8', 'https://watcha.com/ko-KR/contents/m5eJyM9', 'https://watcha.com/ko-KR/contents/m5m1vJx', 'https://watcha.com/ko-KR/contents/m5aVlPj', 'https://watcha.com/ko-KR/contents/myWqxp5', 'https://watcha.com/ko-KR/contents/m5XMqmD', 'https://watcha.com/ko-KR/contents/mOl4YaQ', 'https://watcha.com/ko-KR/contents/mW93opG', 'https://watcha.com/ko-KR/contents/mO1VzlQ', 'https://watcha.com/ko-KR/contents/m5m1vy7', 'https://watcha.com/ko-KR/contents/mdRLwlv', 'https://watcha.com/ko-KR/contents/mpWpZ6d', 'https://watcha.com/ko-KR/contents/mBOkzJW', 'https://watcha.com/ko-KR/contents/mgOAPWm', 'https://watcha.com/ko-KR/contents/md6YxNy', 'https://watcha.com/ko-KR/contents/m5XMv3W', 'https://watcha.com/ko-KR/contents/m5Y6pDl', 'https://watcha.com/ko-KR/contents/mWwBZ9G', 'https://watcha.com/ko-KR/contents/m5xJk3x', 'https://watcha.com/ko-KR/contents/mW93y9B', 'https://watcha.com/ko-KR/contents/mW9EZlW', 'https://watcha.com/ko-KR/contents/m5eJZ3R', 'https://watcha.com/ko-KR/contents/m5YMvR5', 'https://watcha.com/ko-KR/contents/m5GXz42', 'https://watcha.com/ko-KR/contents/mOgBo03', 'https://watcha.com/ko-KR/contents/mOAZ86G', 'https://watcha.com/ko-KR/contents/my5YJEW', 'https://watcha.com/ko-KR/contents/mWJyPpx', 'https://watcha.com/ko-KR/contents/mW93GDZ', 'https://watcha.com/ko-KR/contents/m5n4oYO', 'https://watcha.com/ko-KR/contents/m5rQxJE', 'https://watcha.com/ko-KR/contents/mdMgK6w', 'https://watcha.com/ko-KR/contents/md66PXd', 'https://watcha.com/ko-KR/contents/mWvkX7e', 'https://watcha.com/ko-KR/contents/mWJyL1Q', 'https://watcha.com/ko-KR/contents/mO1V3ok', 'https://watcha.com/ko-KR/contents/m5xJMDr', 'https://watcha.com/ko-KR/contents/mO0gZP1', 'https://watcha.com/ko-KR/contents/mWyJ77M', 'https://watcha.com/ko-KR/contents/mdBzV9D', 'https://watcha.com/ko-KR/contents/mW4LkqY', 'https://watcha.com/ko-KR/contents/mWvkyor', 'https://watcha.com/ko-KR/contents/mO8K0Xb', 'https://watcha.com/ko-KR/contents/m6dRVad', 'https://watcha.com/ko-KR/contents/mW4Lzqg', 'https://watcha.com/ko-KR/contents/m5x6KlO', 'https://watcha.com/ko-KR/contents/mOAZ89E', 'https://watcha.com/ko-KR/contents/mmObrNO', 'https://watcha.com/ko-KR/contents/m5xJXvb', 'https://watcha.com/ko-KR/contents/m5GV6Yd', 'https://watcha.com/ko-KR/contents/m533Z8p', 'https://watcha.com/ko-KR/contents/mdjaLA9', 'https://watcha.com/ko-KR/contents/m45nnX5', 'https://watcha.com/ko-KR/contents/mv5axqd', 'https://watcha.com/ko-KR/contents/mjO0ovW', 'https://watcha.com/ko-KR/contents/m5ZXBo0', 'https://watcha.com/ko-KR/contents/m5xDyj5', 'https://watcha.com/ko-KR/contents/m5aVlap', 'https://watcha.com/ko-KR/contents/m9dBqzO', 'https://watcha.com/ko-KR/contents/mWzwVq9', 'https://watcha.com/ko-KR/contents/mOPoLaP', 'https://watcha.com/ko-KR/contents/mWJykq5', 'https://watcha.com/ko-KR/contents/mdja0A9', 'https://watcha.com/ko-KR/contents/mOl48qL', 'https://watcha.com/ko-KR/contents/mWq9waW', 'https://watcha.com/ko-KR/contents/m5xJngq', 'https://watcha.com/ko-KR/contents/mWLRRDd', 'https://watcha.com/ko-KR/contents/mdjaQyl', 'https://watcha.com/ko-KR/contents/mdEwA1p', 'https://watcha.com/ko-KR/contents/mO8KzlX', 'https://watcha.com/ko-KR/contents/md7DkB5', 'https://watcha.com/ko-KR/contents/mO2M33G', 'https://watcha.com/ko-KR/contents/mNdKveO', 'https://watcha.com/ko-KR/contents/mWLjVZD', 'https://watcha.com/ko-KR/contents/mWJmgx5', 'https://watcha.com/ko-KR/contents/mOl44bQ', 'https://watcha.com/ko-KR/contents/mdMwwJW', 'https://watcha.com/ko-KR/contents/mOAVmGd', 'https://watcha.com/ko-KR/contents/mW4La4l', 'https://watcha.com/ko-KR/contents/mWvk10e', 'https://watcha.com/ko-KR/contents/m5rQQg7', 'https://watcha.com/ko-KR/contents/mO2MKGW', 'https://watcha.com/ko-KR/contents/mdBzQED', 'https://watcha.com/ko-KR/contents/mdEwAR2', 'https://watcha.com/ko-KR/contents/mdEwxpW', 'https://watcha.com/ko-KR/contents/m5QZ8QW', 'https://watcha.com/ko-KR/contents/mOgBxPa', 'https://watcha.com/ko-KR/contents/mOVv6zq', 'https://watcha.com/ko-KR/contents/m5xJLaO', 'https://watcha.com/ko-KR/contents/m5DGE4B', 'https://watcha.com/ko-KR/contents/md7x8N5', 'https://watcha.com/ko-KR/contents/mWqJMnp', 'https://watcha.com/ko-KR/contents/mW9xB85', 'https://watcha.com/ko-KR/contents/m5alkNd', 'https://watcha.com/ko-KR/contents/md768n1', 'https://watcha.com/ko-KR/contents/mOVlo85', 'https://watcha.com/ko-KR/contents/mgdMr45', 'https://watcha.com/ko-KR/contents/m5Y6lKG', 'https://watcha.com/ko-KR/contents/m5nQ4Dd', 'https://watcha.com/ko-KR/contents/mnWJnqW', 'https://watcha.com/ko-KR/contents/my5YvvW', 'https://watcha.com/ko-KR/contents/mWJjpQW', 'https://watcha.com/ko-KR/contents/m5DpxB5', 'https://watcha.com/ko-KR/contents/mWqeQrW', 'https://watcha.com/ko-KR/contents/mWqJvBO', 'https://watcha.com/ko-KR/contents/m5Zx6LO', 'https://watcha.com/ko-KR/contents/mO2lEQ5', 'https://watcha.com/ko-KR/contents/mOVvgo8', 'https://watcha.com/ko-KR/contents/mOAZPad', 'https://watcha.com/ko-KR/contents/m5aNrDW', 'https://watcha.com/ko-KR/contents/mO8XobO', 'https://watcha.com/ko-KR/contents/mpWpow5', 'https://watcha.com/ko-KR/contents/mP5mAx5', 'https://watcha.com/ko-KR/contents/mdEkoAW', 'https://watcha.com/ko-KR/contents/mMO2JAO', 'https://watcha.com/ko-KR/contents/mWpmax4', 'https://watcha.com/ko-KR/contents/mWwVo45', 'https://watcha.com/ko-KR/contents/md7xGv5', 'https://watcha.com/ko-KR/contents/mLOP7lW', 'https://watcha.com/ko-KR/contents/mO8KR42', 'https://watcha.com/ko-KR/contents/mP5m7jO', 'https://watcha.com/ko-KR/contents/mOl4nEg', 'https://watcha.com/ko-KR/contents/m5DGg3d', 'https://watcha.com/ko-KR/contents/m5XMJEA', 'https://watcha.com/ko-KR/contents/mdBzm6G', 'https://watcha.com/ko-KR/contents/mP5mj7W', 'https://watcha.com/ko-KR/contents/m5DVDDO', 'https://watcha.com/ko-KR/contents/mAOl2L5', 'https://watcha.com/ko-KR/contents/mdjagwO', 'https://watcha.com/ko-KR/contents/m5QqYD5', 'https://watcha.com/ko-KR/contents/mOVvJYO', 'https://watcha.com/ko-KR/contents/m5rQG1E', 'https://watcha.com/ko-KR/contents/m5m14vD', 'https://watcha.com/ko-KR/contents/mW939lK', 'https://watcha.com/ko-KR/contents/mWqJnGr', 'https://watcha.com/ko-KR/contents/mWzwV11', 'https://watcha.com/ko-KR/contents/m5mQx35', 'https://watcha.com/ko-KR/contents/mdB61Ad', 'https://watcha.com/ko-KR/contents/m5XML4L', 'https://watcha.com/ko-KR/contents/mY5QNRW', 'https://watcha.com/ko-KR/contents/m5Y0qRd', 'https://watcha.com/ko-KR/contents/mBOkGQO', 'https://watcha.com/ko-KR/contents/mdRLw3r', 'https://watcha.com/ko-KR/contents/mWJJGxW', 'https://watcha.com/ko-KR/contents/mO2MK0n', 'https://watcha.com/ko-KR/contents/mAOlnl5', 'https://watcha.com/ko-KR/contents/m5YrbQW', 'https://watcha.com/ko-KR/contents/mOkXVQd', 'https://watcha.com/ko-KR/contents/md76wkO', 'https://watcha.com/ko-KR/contents/mOA9gEO', 'https://watcha.com/ko-KR/contents/mO8GEjW', 'https://watcha.com/ko-KR/contents/mWpme4M', 'https://watcha.com/ko-KR/contents/mJ5xpq5', 'https://watcha.com/ko-KR/contents/mdjabNO', 'https://watcha.com/ko-KR/contents/mJOVPq5', 'https://watcha.com/ko-KR/contents/mXOgez5', 'https://watcha.com/ko-KR/contents/mYdEogW', 'https://watcha.com/ko-KR/contents/mW4La3e', 'https://watcha.com/ko-KR/contents/md76Zb8', 'https://watcha.com/ko-KR/contents/mdBzV4k', 'https://watcha.com/ko-KR/contents/mdRLx0V', 'https://watcha.com/ko-KR/contents/mWy1NkW', 'https://watcha.com/ko-KR/contents/mAd738d', 'https://watcha.com/ko-KR/contents/m5eJx3g', 'https://watcha.com/ko-KR/contents/mWwBVzq', 'https://watcha.com/ko-KR/contents/m5DGD23', 'https://watcha.com/ko-KR/contents/m5m1lg5', 'https://watcha.com/ko-KR/contents/md76nkO', 'https://watcha.com/ko-KR/contents/mOVv611', 'https://watcha.com/ko-KR/contents/m5NVVwR', 'https://watcha.com/ko-KR/contents/md6xEAW', 'https://watcha.com/ko-KR/contents/mOAZ6Zm', 'https://watcha.com/ko-KR/contents/m5n07VO', 'https://watcha.com/ko-KR/contents/md6k3q5', 'https://watcha.com/ko-KR/contents/mOo0JZd', 'https://watcha.com/ko-KR/contents/mE5D435', 'https://watcha.com/ko-KR/contents/mdEkbEW', 'https://watcha.com/ko-KR/contents/mOgBwNW', 'https://watcha.com/ko-KR/contents/mOg8BMd', 'https://watcha.com/ko-KR/contents/mwWLm0O', 'https://watcha.com/ko-KR/contents/mOkEMz5', 'https://watcha.com/ko-KR/contents/mdjagDO', 'https://watcha.com/ko-KR/contents/mdK2Q9O', 'https://watcha.com/ko-KR/contents/mdKvL7O', 'https://watcha.com/ko-KR/contents/mWJyQM5', 'https://watcha.com/ko-KR/contents/mObGwnd', 'https://watcha.com/ko-KR/contents/mP5mYm5', 'https://watcha.com/ko-KR/contents/mLOPXO6', 'https://watcha.com/ko-KR/contents/mAd7415', 'https://watcha.com/ko-KR/contents/m5nZDV5', 'https://watcha.com/ko-KR/contents/mO2MQNW', 'https://watcha.com/ko-KR/contents/m85XqAW', 'https://watcha.com/ko-KR/contents/mOgBVzW', 'https://watcha.com/ko-KR/contents/mgdM6k5', 'https://watcha.com/ko-KR/contents/mJOV1RW', 'https://watcha.com/ko-KR/contents/mY5Qlyd', 'https://watcha.com/ko-KR/contents/mM5rEdq', 'https://watcha.com/ko-KR/contents/mO2MDxW', 'https://watcha.com/ko-KR/contents/mOPBwV5', 'https://watcha.com/ko-KR/contents/mOVvApO', 'https://watcha.com/ko-KR/contents/mO0gDVW', 'https://watcha.com/ko-KR/contents/mAd7YNW', 'https://watcha.com/ko-KR/contents/mWJarx5', 'https://watcha.com/ko-KR/contents/mBOk88d', 'https://watcha.com/ko-KR/contents/mjO0PzO', 'https://watcha.com/ko-KR/contents/mdEwQ2W', 'https://watcha.com/ko-KR/contents/mWpKxx5', 'https://watcha.com/ko-KR/contents/mZ5e9R5', 'https://watcha.com/ko-KR/contents/m5NMpXd', 'https://watcha.com/ko-KR/contents/mO0rl15', 'https://watcha.com/ko-KR/contents/mW4rmed', 'https://watcha.com/ko-KR/contents/mdMgLx2', 'https://watcha.com/ko-KR/contents/mWqJ28D', 'https://watcha.com/ko-KR/contents/mO2MKxA', 'https://watcha.com/ko-KR/contents/mWLjR9k', 'https://watcha.com/ko-KR/contents/m5NYXpd', 'https://watcha.com/ko-KR/contents/m5rQoaV', 'https://watcha.com/ko-KR/contents/mOoJKyW', 'https://watcha.com/ko-KR/contents/m5NVEE2', 'https://watcha.com/ko-KR/contents/mO8KKXD', 'https://watcha.com/ko-KR/contents/mOPoAqa', 'https://watcha.com/ko-KR/contents/mO0g2Yx', 'https://watcha.com/ko-KR/contents/mOPo6Dl', 'https://watcha.com/ko-KR/contents/mWpkgM5', 'https://watcha.com/ko-KR/contents/mObGGj7', 'https://watcha.com/ko-KR/contents/m5n31DW', 'https://watcha.com/ko-KR/contents/m5Gk725', 'https://watcha.com/ko-KR/contents/mdK3Qkd', 'https://watcha.com/ko-KR/contents/mdReBLO', 'https://watcha.com/ko-KR/contents/mOANbld', 'https://watcha.com/ko-KR/contents/mWyP8Y5', 'https://watcha.com/ko-KR/contents/m5ZXX2e', 'https://watcha.com/ko-KR/contents/m5QqDl5', 'https://watcha.com/ko-KR/contents/md76veO', 'https://watcha.com/ko-KR/contents/mOP7XpW', 'https://watcha.com/ko-KR/contents/m5aVNpW', 'https://watcha.com/ko-KR/contents/m5GwDD5', 'https://watcha.com/ko-KR/contents/mdE96Vd', 'https://watcha.com/ko-KR/contents/mOglQMd', 'https://watcha.com/ko-KR/contents/m5DGX1d', 'https://watcha.com/ko-KR/contents/mWyJYnx', 'https://watcha.com/ko-KR/contents/mWyJpxy', 'https://watcha.com/ko-KR/contents/mWpmnxO', 'https://watcha.com/ko-KR/contents/mdKB79y', 'https://watcha.com/ko-KR/contents/mWvkD0D', 'https://watcha.com/ko-KR/contents/mOlgPnW', 'https://watcha.com/ko-KR/contents/md76Pl8', 'https://watcha.com/ko-KR/contents/mdjal9O', 'https://watcha.com/ko-KR/contents/mO2MvNW', 'https://watcha.com/ko-KR/contents/m5mpXAd', 'https://watcha.com/ko-KR/contents/mOPoaXW', 'https://watcha.com/ko-KR/contents/mOk66kW', 'https://watcha.com/ko-KR/contents/mr5ZGLW', 'https://watcha.com/ko-KR/contents/m5Q4mGd', 'https://watcha.com/ko-KR/contents/m53MkyO', 'https://watcha.com/ko-KR/contents/mWzVYBW', 'https://watcha.com/ko-KR/contents/m5mQaj5', 'https://watcha.com/ko-KR/contents/mWyJ1v6', 'https://watcha.com/ko-KR/contents/md7ZBBO', 'https://watcha.com/ko-KR/contents/m5nQlkd', 'https://watcha.com/ko-KR/contents/mP5mD3d', 'https://watcha.com/ko-KR/contents/mdM3emW', 'https://watcha.com/ko-KR/contents/mdBzn2q', 'https://watcha.com/ko-KR/contents/mAd7yPW', 'https://watcha.com/ko-KR/contents/mWqn7XO', 'https://watcha.com/ko-KR/contents/mVWz0Pd', 'https://watcha.com/ko-KR/contents/m5rokDW', 'https://watcha.com/ko-KR/contents/mOPBqw5', 'https://watcha.com/ko-KR/contents/m533Lw5', 'https://watcha.com/ko-KR/contents/mOA9xlO', 'https://watcha.com/ko-KR/contents/mWpbEAd', 'https://watcha.com/ko-KR/contents/mdj1aJO', 'https://watcha.com/ko-KR/contents/mNdKPR5', 'https://watcha.com/ko-KR/contents/mdRKAD5', 'https://watcha.com/ko-KR/contents/mO0gZ8X', 'https://watcha.com/ko-KR/contents/m5X3JLd', 'https://watcha.com/ko-KR/contents/mWz6MBW', 'https://watcha.com/ko-KR/contents/mWpmmYK', 'https://watcha.com/ko-KR/contents/m5D2MDO', 'https://watcha.com/ko-KR/contents/mO2aDaW', 'https://watcha.com/ko-KR/contents/mOgV7AW', 'https://watcha.com/ko-KR/contents/m5xYKqW', 'https://watcha.com/ko-KR/contents/mdR0brW', 'https://watcha.com/ko-KR/contents/mdBzma9', 'https://watcha.com/ko-KR/contents/mDWvrxW', 'https://watcha.com/ko-KR/contents/mdEw9nb', 'https://watcha.com/ko-KR/contents/myWqkld', 'https://watcha.com/ko-KR/contents/mdMgLRW', 'https://watcha.com/ko-KR/contents/m5Xxqp5', 'https://watcha.com/ko-KR/contents/mAd7avW', 'https://watcha.com/ko-KR/contents/mP5mQx5', 'https://watcha.com/ko-KR/contents/md7q6MW', 'https://watcha.com/ko-KR/contents/mQO8YGd', 'https://watcha.com/ko-KR/contents/mOk6qjW', 'https://watcha.com/ko-KR/contents/mwWLKz5', 'https://watcha.com/ko-KR/contents/mdRLrvW', 'https://watcha.com/ko-KR/contents/m2djqJO', 'https://watcha.com/ko-KR/contents/mWJyv45', 'https://watcha.com/ko-KR/contents/mnWJ2QW', 'https://watcha.com/ko-KR/contents/mdja6rO', 'https://watcha.com/ko-KR/contents/mBOkqE5', 'https://watcha.com/ko-KR/contents/mLOPkXW', 'https://watcha.com/ko-KR/contents/my5YGwO', 'https://watcha.com/ko-KR/contents/m45ngDW', 'https://watcha.com/ko-KR/contents/m1WwXlO', 'https://watcha.com/ko-KR/contents/mP5mPzd', 'https://watcha.com/ko-KR/contents/mr5ZNK5', 'https://watcha.com/ko-KR/contents/mO8Zj35', 'https://watcha.com/ko-KR/contents/mO0glzW', 'https://watcha.com/ko-KR/contents/mdKB7PJ', 'https://watcha.com/ko-KR/contents/mOAR0Pd', 'https://watcha.com/ko-KR/contents/m5XBN4O', 'https://watcha.com/ko-KR/contents/md6YZaO', 'https://watcha.com/ko-KR/contents/mdEAKAd', 'https://watcha.com/ko-KR/contents/mW4alwO', 'https://watcha.com/ko-KR/contents/m5X3Rgd', 'https://watcha.com/ko-KR/contents/mWJy3J5', 'https://watcha.com/ko-KR/contents/mW4AagO', 'https://watcha.com/ko-KR/contents/mdBYBYO', 'https://watcha.com/ko-KR/contents/m5ejMyW', 'https://watcha.com/ko-KR/contents/mY5Q0Jd', 'https://watcha.com/ko-KR/contents/mOAZkQd', 'https://watcha.com/ko-KR/contents/m5nQMnd', 'https://watcha.com/ko-KR/contents/mgOARjd', 'https://watcha.com/ko-KR/contents/m5xJqxO', 'https://watcha.com/ko-KR/contents/myWqyBW', 'https://watcha.com/ko-KR/contents/mWpmAMO', 'https://watcha.com/ko-KR/contents/mM5rmEd', 'https://watcha.com/ko-KR/contents/mWJvZRW', 'https://watcha.com/ko-KR/contents/mMO2mNW', 'https://watcha.com/ko-KR/contents/mOoyZZ5', 'https://watcha.com/ko-KR/contents/mWwBQl5', 'https://watcha.com/ko-KR/contents/m5xMKrW', 'https://watcha.com/ko-KR/contents/m5QjklW', 'https://watcha.com/ko-KR/contents/m5ZxBRO', 'https://watcha.com/ko-KR/contents/m5eJP3W', 'https://watcha.com/ko-KR/contents/mVWzX9d', 'https://watcha.com/ko-KR/contents/mO00RLO', 'https://watcha.com/ko-KR/contents/md7ZPBO', 'https://watcha.com/ko-KR/contents/mWz4Z0d', 'https://watcha.com/ko-KR/contents/mXOgm05', 'https://watcha.com/ko-KR/contents/mOPo3VW', 'https://watcha.com/ko-KR/contents/mdEjNm5', 'https://watcha.com/ko-KR/contents/mW93M4d', 'https://watcha.com/ko-KR/contents/mmObLL5', 'https://watcha.com/ko-KR/contents/mr5Z8RO', 'https://watcha.com/ko-KR/contents/md6Y82O', 'https://watcha.com/ko-KR/contents/m5ZXwLO', 'https://watcha.com/ko-KR/contents/mE5DzQO', 'https://watcha.com/ko-KR/contents/mdR7xv5', 'https://watcha.com/ko-KR/contents/mmOb3kd', 'https://watcha.com/ko-KR/contents/m5XMeBW', 'https://watcha.com/ko-KR/contents/m5eQyz5', 'https://watcha.com/ko-KR/contents/mOAD1EW', 'https://watcha.com/ko-KR/contents/m5NvJX5', 'https://watcha.com/ko-KR/contents/mJOVXR5', 'https://watcha.com/ko-KR/contents/m5m14z5', 'https://watcha.com/ko-KR/contents/mQO8leO', 'https://watcha.com/ko-KR/contents/mQO8nX5', 'https://watcha.com/ko-KR/contents/mOo0Pzd', 'https://watcha.com/ko-KR/contents/mY5QVRW', 'https://watcha.com/ko-KR/contents/mdEww2W', 'https://watcha.com/ko-KR/contents/mLOPPPO', 'https://watcha.com/ko-KR/contents/m5Y6JvW', 'https://watcha.com/ko-KR/contents/m5xjRgO', 'https://watcha.com/ko-KR/contents/mO8jDq5', 'https://watcha.com/ko-KR/contents/mWyzmNO', 'https://watcha.com/ko-KR/contents/m5xQEqW', 'https://watcha.com/ko-KR/contents/mdjpxwW', 'https://watcha.com/ko-KR/contents/mLOPA7d', 'https://watcha.com/ko-KR/contents/m5X6o4O', 'https://watcha.com/ko-KR/contents/m5Y63jW', 'https://watcha.com/ko-KR/contents/mVWzB0W', 'https://watcha.com/ko-KR/contents/mOPopwW', 'https://watcha.com/ko-KR/contents/mWvkREO', 'https://watcha.com/ko-KR/contents/mObMNNW', 'https://watcha.com/ko-KR/contents/mWpkJw5', 'https://watcha.com/ko-KR/contents/mdKg2Rd', 'https://watcha.com/ko-KR/contents/m5Nla2O', 'https://watcha.com/ko-KR/contents/mdjlyzW', 'https://watcha.com/ko-KR/contents/m6dR6gd', 'https://watcha.com/ko-KR/contents/mQO8XVO', 'https://watcha.com/ko-KR/contents/m5Zo00W', 'https://watcha.com/ko-KR/contents/mYdE0OX', 'https://watcha.com/ko-KR/contents/mOAwoP5', 'https://watcha.com/ko-KR/contents/mYdE0EO', 'https://watcha.com/ko-KR/contents/mJOVBZO', 'https://watcha.com/ko-KR/contents/mO0mxad', 'https://watcha.com/ko-KR/contents/mOAVQZd', 'https://watcha.com/ko-KR/contents/m5awavO', 'https://watcha.com/ko-KR/contents/mW9rPKO', 'https://watcha.com/ko-KR/contents/mOgmjz5', 'https://watcha.com/ko-KR/contents/m5GwnB5', 'https://watcha.com/ko-KR/contents/mWp1EmO', 'https://watcha.com/ko-KR/contents/mO1QooO', 'https://watcha.com/ko-KR/contents/m5rgjEW', 'https://watcha.com/ko-KR/contents/m5ewEZO', 'https://watcha.com/ko-KR/contents/mObGNRd', 'https://watcha.com/ko-KR/contents/mn5NYnd', 'https://watcha.com/ko-KR/contents/mOV6XzO', 'https://watcha.com/ko-KR/contents/m5QbDRW', 'https://watcha.com/ko-KR/contents/mO2rjNO', 'https://watcha.com/ko-KR/contents/mOAw9G5', 'https://watcha.com/ko-KR/contents/mOkE1g5', 'https://watcha.com/ko-KR/contents/mWv1lb5', 'https://watcha.com/ko-KR/contents/mO2N00O', 'https://watcha.com/ko-KR/contents/mOVgp1d', 'https://watcha.com/ko-KR/contents/m5GvRKd', 'https://watcha.com/ko-KR/contents/mOAgnP5', 'https://watcha.com/ko-KR/contents/m5Dk2Rd', 'https://watcha.com/ko-KR/contents/mW4LMvL', 'https://watcha.com/ko-KR/contents/m5xY3qW', 'https://watcha.com/ko-KR/contents/mWpme6g', 'https://watcha.com/ko-KR/contents/mdMggKy', 'https://watcha.com/ko-KR/contents/mOPoEMa', 'https://watcha.com/ko-KR/contents/mdMgAmm', 'https://watcha.com/ko-KR/contents/mWzwo84', 'https://watcha.com/ko-KR/contents/m5nQxkn', 'https://watcha.com/ko-KR/contents/mWyQz6W', 'https://watcha.com/ko-KR/contents/m5ZXvVK', 'https://watcha.com/ko-KR/contents/m5mGVzO', 'https://watcha.com/ko-KR/contents/mWJyZ83', 'https://watcha.com/ko-KR/contents/m5rLzEO', 'https://watcha.com/ko-KR/contents/m5eJBq9', 'https://watcha.com/ko-KR/contents/mWzM9D5', 'https://watcha.com/ko-KR/contents/mQO8LqO', 'https://watcha.com/ko-KR/contents/mOAZxQd', 'https://watcha.com/ko-KR/contents/mYdEnE5', 'https://watcha.com/ko-KR/contents/mWLar6O', 'https://watcha.com/ko-KR/contents/mOPpNlO', 'https://watcha.com/ko-KR/contents/m5ZXrxO', 'https://watcha.com/ko-KR/contents/mOAPbEW', 'https://watcha.com/ko-KR/contents/mOgmjY5', 'https://watcha.com/ko-KR/contents/m5e8v3d', 'https://watcha.com/ko-KR/contents/mO2x9k5', 'https://watcha.com/ko-KR/contents/mdE09AO', 'https://watcha.com/ko-KR/contents/mW4L2XW', 'https://watcha.com/ko-KR/contents/m5rQoRd', 'https://watcha.com/ko-KR/contents/mOPooYW', 'https://watcha.com/ko-KR/contents/m5mQ9z5', 'https://watcha.com/ko-KR/contents/m5xNMjO', 'https://watcha.com/ko-KR/contents/m5NVoR5', 'https://watcha.com/ko-KR/contents/mdKmrJW', 'https://watcha.com/ko-KR/contents/mBOkBwW', 'https://watcha.com/ko-KR/contents/mjO0k7W', 'https://watcha.com/ko-KR/contents/m5nEZqd', 'https://watcha.com/ko-KR/contents/md76JPO', 'https://watcha.com/ko-KR/contents/m5n80DO', 'https://watcha.com/ko-KR/contents/m5NEynd', 'https://watcha.com/ko-KR/contents/m6dRLLW', 'https://watcha.com/ko-KR/contents/mW98ZM5', 'https://watcha.com/ko-KR/contents/mW4XgqW', 'https://watcha.com/ko-KR/contents/m5XXNL5', 'https://watcha.com/ko-KR/contents/m85X6BO', 'https://watcha.com/ko-KR/contents/mdja2NO', 'https://watcha.com/ko-KR/contents/mdMgZkW', 'https://watcha.com/ko-KR/contents/m533kl5', 'https://watcha.com/ko-KR/contents/mJ5x8x5', 'https://watcha.com/ko-KR/contents/m5e77oO', 'https://watcha.com/ko-KR/contents/mP5mgmO', 'https://watcha.com/ko-KR/contents/mW9GGZd', 'https://watcha.com/ko-KR/contents/mAd7k2d', 'https://watcha.com/ko-KR/contents/mWvkKMO', 'https://watcha.com/ko-KR/contents/mDWv6bW', 'https://watcha.com/ko-KR/contents/mW9208O', 'https://watcha.com/ko-KR/contents/m5rND75', 'https://watcha.com/ko-KR/contents/mWz4g4d', 'https://watcha.com/ko-KR/contents/m6dRLWy', 'https://watcha.com/ko-KR/contents/m15G9NO', 'https://watcha.com/ko-KR/contents/m2dj6Jd', 'https://watcha.com/ko-KR/contents/mOP73PW', 'https://watcha.com/ko-KR/contents/m5aVQDW', 'https://watcha.com/ko-KR/contents/md6xaaW', 'https://watcha.com/ko-KR/contents/m5Gvy2d', 'https://watcha.com/ko-KR/contents/mnWJ1n5', 'https://watcha.com/ko-KR/contents/mVWzvZW', 'https://watcha.com/ko-KR/contents/mWvDGZd', 'https://watcha.com/ko-KR/contents/mgOA3E5', 'https://watcha.com/ko-KR/contents/m5nGp9W', 'https://watcha.com/ko-KR/contents/mjO0leO', 'https://watcha.com/ko-KR/contents/mWpJoad', 'https://watcha.com/ko-KR/contents/mJ5xvEO', 'https://watcha.com/ko-KR/contents/mWqJBpO', 'https://watcha.com/ko-KR/contents/md7662O']

In [128]:
data_watcha=pd.DataFrame(columns=['id','title', 'title_ko','rate', 'year','country','genre','running_time','contents'])
watcha_review=pd.DataFrame(columns=['movie_id','user','rate','likes','review'])

In [5]:
data_watcha = pd.read_csv('data_watcha.csv')
watcha_review = pd.read_csv('watcha_review.csv')

In [12]:
data_watcha.to_csv('data_watcha.csv')
watcha_review.to_csv('watcha_review.csv')

각 영화에서 영화의 정보/장르/리뷰를 가져온다

In [9]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('lang=ko_KR')

for i in tqdm(range(0,len(movies))):
    #movie
    page=requests.get(movies[i])
    html=page.text
    soup=BeautifulSoup(html, 'html.parser')
    title_ko=soup.select('h1.css-13a04pq-Title.e1svyhwg14')[0].text
    rate_raw=soup.select('span.css-1r45xsg-RatingStatHeaderTopRight.e1dk30cn0')[0].text
    rate=rate_raw[rate_raw.find("★")+1:rate_raw.find("(")]
    
    #movie overview
    page=requests.get(movies[i]+"/overview")
    html=page.text
    soup=BeautifulSoup(html, 'html.parser')
    overview=soup.select('dl dd')
    
    title = overview[0].text
    year = overview[1].text
    country = overview[2].text
    genre = overview[3].text
    running_time = overview[4].text
    contents = overview[5].text

    #movie comment
    driver = webdriver.Chrome('./chromedriver', options=chrome_options)
    driver.implicitly_wait(3)

    driver.get(movies[i]+"/comments")

    SCROLL_PAUSE_TIME=2.5
    last_height = driver.execute_script("return document.body.scrollHeight")

    scroll_num = 0

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")

        if scroll_num == 6:
            break

        if new_height == last_height:
            break

        scroll_num += 1
        last_height = new_height

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    review_list=soup.select('div.css-wnwcvo-Comment.e1oskw6f0')
    for review in review_list:
        try:
            user=review.select('a.css-1f9m1s4-StylelessLocalLink.e9fw3ul1')[0].text.replace(" ","")
            rate=review.select('div.css-1eyufz5-UserActionStatus.e1oskw6f4 span')[0].text
            try:
                likes=review.select('div.css-olvdbz-LikeReplyBlock.e1xxz10x1 em')[0].text
            except:
                likes=0.0
            try:
                comment=review.select('div.css-o67ghx-Text-handleRenderInner.e1xxz10x0')[0].text
            except:
                comment=review.select('div.css-1n96fhq-Text-handleRenderInner.e1xxz10x0')[0].text
            temp=pd.Series([i,user,rate,likes,comment],index=['movie_id','user','rate','likes','review'])
            watcha_review=watcha_review.append(temp,ignore_index=True)
        except:
            pass
        
    driver.close()
    
    temp=pd.Series([i,title,title_ko,rate,year,country,genre,running_time,contents],
                   index=['id','title', 'title_ko','rate', 'year','country','genre','running_time','contents'])
    data_watcha=data_watcha.append(temp,ignore_index=True)

In [10]:
data_watcha

,Unnamed: 0,id,title,title_ko,rate,year,country,genre,running_time,contents
0,0.0,0,김씨 표류기,김씨 표류기,4.0,2009,한국,드라마/코미디,1시간 56분,자살시도가 실패로 끝나 한강의 밤섬에 불시착한 남자. 죽는 것도 쉽지 않자 일단 섬...
1,1.0,1,The Girl on the Train,걸 온 더 트레인,2.5,2016,미국,범죄/드라마/미스터리/스릴러,1시간 52분,이혼 후 알코올중독에 빠져 심각한 블랙아웃 (음주 후 필름 끊기는 현상) 을 겪고 ...
2,2.0,2,Ένας Άλλος Κόσμος,"나의 사랑, 그리스",4.0,2015,그리스,드라마/로맨스,1시간 54분,그리스를 배경으로 펼쳐지는 각기 다른 세대 세 커플의 사랑 이야기. [부메랑] 여대...
3,3.0,3,LA CARA OCULTA,히든 페이스,3.0,2011,"콜롬비아,스페인",코미디/드라마/미스터리/스릴러,1시간 37분,오케스트라 지휘자 아드리안은 1년간 콜롬비아의 보고타 필하모닉의 지휘자로 일하게되어...
4,4.0,4,Deepwater Horizon,딥워터 호라이즌,3.5,2016,"홍콩,미국",액션/드라마/역사/스릴러/재난,1시간 47분,"2010년 4월 20일, 미국 루이지애나주 앞바다 멕시코만 석유 시추선 ‘딥워터 호..."
...,...,...,...,...,...,...,...,...,...,...
666,NaN,665,Pirates of the Caribbean: The Curse of the Bla...,캐리비안의 해적 - 블랙 펄의 저주,3.0,2003,미국,액션/판타지/모험,2시간 23분,매력 넘치는 해적 캡틴 잭 스패로우(죠니 뎁 분)에게 수정처럼 맑고 투명한 카리브 ...
667,NaN,666,Thor,토르: 천둥의 신,3.0,2011,미국,액션/모험/판타지/SF,1시간 52분,신의 세계 ‘아스가르드’의 후계자로 강력한 파워를 지닌 천둥의 신 ‘토르’. 평소 ...
668,NaN,667,不能説的・秘密,말할 수 없는 비밀,3.0,2007,대만,로맨스/판타지/드라마,1시간 41분,예술학교로 전학 온 상륜(주걸륜)은 아버지의 영향을 받아 피아노에 천부적인 소질을 ...
669,NaN,668,Harry Potter and the Prisoner of Azkaban,해리포터와 아즈카반의 죄수,4.0,2004,"영국,미국",판타지/미스터리/가족/모험/액션,2시간 21분,13세가 된 해리 포터(다니엘 래드클리프)는 또 한번의 여름 방학을 이모 가족인 더...


In [11]:
watcha_review

,Unnamed: 0,movie_id,user,rate,likes,review
0,0.0,0,최한영,4.5,656,뜨지 못한 명작
1,1.0,0,seulkiki,5.0,575,사회라는 거대한 틀에서 표류하고있는 두 사람의 극복기. 정말괜찮은 영화.
2,2.0,0,DamianJeonDongHyun,4.0,360,정말 정재영의 연기중 손에 꼽을 만한 작품이라고 말해주고 싶다.\r\n실제 사회에서...
3,3.0,0,새별님,5.0,311,나는 이런 영화가 좋더라..감독은 틀림없이 따뜻한 사람일 것이다
4,4.0,0,BB,5.0,301,이 영화에서 가장 난해한건 바로 포스터ㅋ
...,...,...,...,...,...,...
29678,NaN,669,챠챠,4.0,15,한국형 로코물 중 베스트 안에 드는 것 같다.
29679,NaN,669,배소이,3.0,14,어린신부에 이어 재평가 받는다면 살아남지 못할 영화.\n시대를 잘 타서 레전드로 불...
29680,NaN,669,태양계,5.0,14,"소설도 좋았고 영화도 좋았다. 오래전에 봤지만, 전지현이 차태현을 건너편 산에 오르..."
29681,NaN,669,Jaewon,5.0,13,캐논을 좋아하게 된 이유
